<a href="https://colab.research.google.com/github/AAhmadS/deeplearning-HW4/blob/main/NLP_task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers
!pip install torchtext==0.5.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd
import numpy as np
from numpy import pad
import matplotlib.pyplot as plt
plt.switch_backend('agg')
%matplotlib inline
import matplotlib.ticker as ticker
import torch
from torch import nn
import torchtext
from torchtext.vocab import Vocab
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, TensorDataset
from collections import Counter
from torchtext.data.utils import get_tokenizer
import nltk
from nltk.tokenize import sent_tokenize
from transformers import pipeline, AutoTokenizer
from __future__ import unicode_literals, print_function, division
from torch import optim
import torch.nn.functional as F
import random

from tqdm import tqdm

#Section 1 : Loading data and preprocessing


In [5]:
tokenizer = AutoTokenizer.from_pretrained('bolbolzaban/gpt2-persian')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
with open("ferdousi.txt","r") as fle:
  file = fle.read().splitlines()[2:]

In [7]:
sentences = [tokenizer.tokenize(x) for x in file]

In [8]:
tokenized = [tokenizer.convert_tokens_to_ids(x) for x in sentences]

In [9]:
sentences[0]

['▁به', '▁نام', '▁خداوند', '▁جان', '▁و', '▁خرد']

In [10]:
max_len=0
max_num=0
for x in tokenized:
  if(max_len<len(x)):max_len = len(x)
  if(max(x)>max_num):
    max_num = max(x)
print(max_len, max_num)

14 24999


In [11]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {'[BOP]':10, '<unk>':0, tokenizer.eos_token : tokenizer.eos_token_id, tokenizer.bos_token : tokenizer.bos_token_id}
        self.word2count = {}
        self.index2word = {tokenizer.bos_token_id: tokenizer.bos_token, tokenizer.eos_token_id: tokenizer.eos_token, 10:'[BOP]', 0:'<unk>'}
        self.n_words = 25000 

    def addSentence(self, sentences, tokenized):
        for i in range(len(sentences)):
          for j in range(len(sentences[i])):
            self.addWord(sentences[i][j],tokenized[i][j])

    def addWord(self, word, index):
        if word not in self.word2index:
            self.word2index[word] = index
            self.word2count[word] = 1
            self.index2word[index] = word
        else:
            self.word2count[word] += 1

In [12]:
po_persian = Lang("persian")
po_persian.addSentence(sentences,tokenized)

In [13]:
input_lang = po_persian
output_lang = po_persian
SB_token = '[BOP]'
SB_token_id = 10
WSpace_token_id=0
sentences
tokenized
Global_pad_arg = 14

def indexesFromSentence(lang, sentence_ind):
    return [lang.word2index[word] for word in sentences[sentence_ind]]


def tensorFromSentence(lang, sentence_ind, pad_arg, code=0):
    indexes = indexesFromSentence(lang, (sentence_ind)%99217)
    while len(indexes)<pad_arg:
      indexes.append(WSpace_token_id)
    if code ==1 :
      indexes.append(tokenizer.eos_token_id)
    else:
      indexes.append(SB_token_id)
    return torch.tensor(indexes, dtype=torch.int32).view(-1, 1)


def tensorsFromPair(pair_ind):
    input_tensor = torch.cat((tensorFromSentence(input_lang, 2*pair_ind, 14),tensorFromSentence(input_lang,2*pair_ind+1,14,code=1)))
    target_tensor = torch.cat((tensorFromSentence(output_lang,2*pair_ind+2, 14),tensorFromSentence(output_lang, 2*pair_ind+3, 14,code=1)))
    return np.array(input_tensor), np.array(target_tensor)

In [14]:
train_data = np.array([tensorsFromPair(x) for x in range(49608)])

In [15]:
X = train_data[:,0,:,:]
y = train_data[:,1,:,:]

X = torch.from_numpy(X)
y = torch.from_numpy(y)

train_dataset = TensorDataset(X, y)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

#Section 2 : LSTM model

In [16]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')

Using cuda for inference


In [17]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(dim = 1, keepdim=True).reshape(y.size())
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc,top_pred

In [18]:
from logging import raiseExceptions
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout, batch_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=num_layers, dropout=dropout, batch_first=True)

    def forward(self, input, hidden, cell):
        embedded = self.embedding(input).view(input.size()[0], input.size()[1], -1)
        output, (hidden, cell) = self.lstm(embedded, (hidden,cell))
        return output, hidden, cell

    def initHidden(self):
        hidden = torch.zeros(self.num_layers, self.batch_size, self.hidden_size).to(device)
        cell = torch.zeros(self.num_layers, self.batch_size, self.hidden_size).to(device)
        return hidden, cell



class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers, dropout,batch_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=num_layers, dropout=dropout, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, cell):
        output = self.embedding(input).view(input.size()[0], input.size()[1], -1)
        output = F.relu(output)
        output, (hidden, cell) = self.lstm(output, (hidden,cell))
        output = self.softmax(self.out(output[0]))
        return output, hidden , cell

    def initHidden(self):
        hidden = torch.zeros(self.num_layers, self.batch_size, self.hidden_size).to(device)
        cell = torch.zeros(self.num_layers, self.batch_size, self.hidden_size).to(device)
        return hidden, cell

In [ ]:
def trainIters(encoder, decoder, epochs, encoder_optimizer, decoder_optimizer, train_dataloader, criterion):
  target_length=30
  losses = []
  accs = []
  encoder.train()
  decoder.train()

  encoder_optimizer.zero_grad()
  decoder_optimizer.zero_grad()

  for epoch in range(epochs):
      hidden, cell = encoder.initHidden()

      epoch_loss = 0
      accuracy = 0
      with tqdm(enumerate(train_dataloader),total=len(train_dataloader)) as pbar:
        for i,(X, y) in pbar:
            X = X.to(device)
            y = y.to(device)

            #encoder step

            output, hidden, cell = encoder(X, hidden, cell)

            #decoder step
            
            decoder_input = torch.ones_like(torch.empty(X.size()[0],1,1),device=device).to(torch.int)*tokenizer.bos_token_id
            loss = 0
            for di in range(target_length):
              decoder_output, hidden, cell = decoder(decoder_input, hidden,cell)
              print(decoder_output.size())
              loss += criterion(decoder_output, y[:,di])
              #acc , _ = calculate_accuracy(output, y[:][di])
              decoder_input = y[:][di]
            output, hidden, cell = decoder(decoder_input, hidden , cell)

            #loss and accuracy

            
            #accuracy += acc
            epoch_loss += loss
            loss.backward()
            
            #optimizer steps

            encoder_optimizer.step()
            decoder_optimizer.step()

            pbar.set_description(f'EPOCH:{epoch+1} - Loss: {epoch_loss/(i+1):.4f} - Accuracy: {accuracy*100/(i+1):.4f}%')

        losses.append((epoch_loss/len(pbar)).cpu().item())
        accs.append((accuracy*100/len(pbar)).cpu().item())
        if (losses[-1]==min(losses)):
            torch.save(encoder.state_dict(), 'Ferdousi_encoder.pth')
            torch.save(decoder.state_dict(), 'Ferdousi_decoder.pth')


In [ ]:
hidden_size = 128
learning_rate = 1e-2

encoder1 = EncoderRNN(input_lang.n_words, hidden_size, 2, 0.15,64).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words, 2, 0.15,64).to(device)

encoder_optimizer = optim.Adam(encoder1.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder1.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

trainIters(encoder1, decoder1, 5,encoder_optimizer, decoder_optimizer, train_dataloader, criterion)

  0%|          | 0/776 [00:00<?, ?it/s]

torch.Size([1, 25000])


ValueError: ignored

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence,14)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[BOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input,decoder_hidden)
            _, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [ ]:
evaluate(encoder1,decoder1,0,max_length = 30)

In [ ]:
def sentencer(decoded):
  sen = ""
  for x in range(len(decoded)-1):
    sen = sen+""+decoded[x]

In [ ]:
listt = evaluate(encoder1,decoder1,10,200)
tokenizer.encode(listt[2])

In [ ]:
MAX_LENGTH = 30
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[tokenizer.bos_token_id]], device=device)

    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(
            decoder_input, decoder_hidden)
        loss += criterion(decoder_output, target_tensor[di])
        decoder_input = target_tensor[di]
    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length


def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    plot_losses = []
    print_loss_total = 0
    plot_loss_total = 0

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)

    training_pairs = [tensorsFromPair(random.choice(np.arange(49607))) for i in range(n_iters)]
    criterion = nn.CrossEntropyLoss()

    for iter in tqdm(range(1, n_iters + 1)):
        training_pair = training_pairs[iter - 1]

        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
          print_loss_avg = print_loss_total / print_every
          print_loss_total = 0
          print('%s (%d%%) %.4f' % (iter, iter / n_iters * 100, print_loss_avg))

        # #if iter % plot_every == 0:
        # plot_loss_avg = plot_loss_total / plot_every
        # plot_losses.append(plot_loss_avg)
        # plot_loss_total = 0

    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(plot_losses)